In [1]:
import numpy as np
import random
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [5]:
def cargar_datos(nombre_archivo):
    """
    Loads the Uncapacitated Facility Location Problem instance from a text file.

    Args:
        nombre_archivo (str): The path to the input file.

    Returns:
        tuple: A tuple containing:
            n (int): Number of facilities.
            m (int): Number of clients.
            f_j (list): List of fixed costs for each facility (index 1 to n).
            c_ij (list of lists): 2D list of assignment costs from client i to facility j
                                  (indices 1 to m for clients, 1 to n for facilities).
    """
    with open(nombre_archivo, 'r') as f:
        lineas = f.readlines()

    n, m = map(int, lineas[0].split())

    f_j = [0] * (n + 1)
    c_ij = [[0] * (n + 1) for _ in range(m + 1)]

    for j in range(1, n + 1):
        datos = list(map(int, lineas[j].split()))
        idx = datos[0]
        f_j[idx] = datos[1]
        for i in range(1, m + 1):
            c_ij[i][idx] = datos[i + 1]
    return n, m, f_j, c_ij

In [12]:
class UFLP_GA:
    """
    A Genetic Algorithm implementation for the Uncapacitated Facility Location Problem (UFLP).
    """

    def __init__(self, n_facilities, n_customers, fixed_costs, transport_costs,
                 population_size=50, mutation_rate=0.1, crossover_rate=0.8,
                 tournament_size=3, max_generations=100, crossover_type='uniform'):
        """
        Initializes the UFLP_GA.

        Args:
            n_facilities (int): The number of potential facility locations.
            n_customers (int): The number of customers to serve.
            fixed_costs (np.array): A 1D array of fixed costs for opening each facility.
            transport_costs (np.array): A 2D array where transport_costs[i][j] is the cost
                                       of transporting goods from facility i to customer j.
            population_size (int): The number of individuals in the population.
            mutation_rate (float): The probability of a gene being mutated.
            crossover_rate (float): The probability of two parents undergoing crossover.
            tournament_size (int): The number of individuals participating in tournament selection.
            max_generations (int): The maximum number of generations to run the algorithm.
        """
        self.n_facilities = n_facilities
        self.n_customers = n_customers
        self.fixed_costs = np.array(fixed_costs[1:])  # Ignore index 0
        self.transport_costs = np.array(transport_costs[1:])[:, 1:]  # Ignore index 0
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.tournament_size = tournament_size
        self.max_generations = max_generations
        self.crossover_type = crossover_type  # Store crossover type
        self.population = []
        self.generation = 0
        self.best_solution = None
        self.best_fitness = float('inf')
        self.fitness_history = []

    def _initialize_population(self):
        """
        Initializes the population with random binary strings.
        """
        self.population = [self._generate_individual() for _ in range(self.population_size)]
        self._evaluate_population()
        self.population.sort(key=lambda ind: ind['fitness'])
        self.best_solution = self.population[0]['solution'].copy()
        self.best_fitness = self.population[0]['fitness']
        self.fitness_history.append(self.best_fitness)

    def _generate_individual(self):
        """
        Generates a random binary string representing a solution.
        """
        return {'solution': np.random.randint(0, 2, self.n_facilities), 'fitness': None}

    def _calculate_fitness(self, individual):
        """
        Calculates the total cost of a solution.

        Args:
            individual (dict): A dictionary containing the 'solution' (binary array) and 'fitness'.

        Returns:
            float: The total cost of the solution.
        """
        open_facilities_indices = np.where(individual['solution'] == 1)[0]
        if not open_facilities_indices.size:
            return float('inf')  # Infeasible: no open facilities

        fixed_cost = np.sum(self.fixed_costs[open_facilities_indices])
        transportation_cost = 0

        for i in range(self.n_customers):
            min_cost = float('inf')
            for j in open_facilities_indices:
                cost = self.transport_costs[i][j]
                if cost < min_cost:
                    min_cost = cost
            transportation_cost += min_cost

        total_cost = fixed_cost + transportation_cost
        return total_cost

    def _evaluate_population(self):
        """
        Evaluates the fitness of each individual in the population.
        """
        for individual in self.population:
            individual['fitness'] = self._calculate_fitness(individual)
            if individual['fitness'] < self.best_fitness:
                self.best_fitness = individual['fitness']
                self.best_solution = individual['solution'].copy()

    def _tournament_selection(self):
        """
        Selects a parent using tournament selection.
        """
        tournament = random.sample(self.population, self.tournament_size)
        winner = min(tournament, key=lambda ind: ind['fitness'])
        return winner

    def _uniform_crossover(self, parent1, parent2):
        """
        Performs uniform crossover between two parents and returns two offspring.
        """
        offspring1_solution = np.empty(self.n_facilities, dtype=int)
        offspring2_solution = np.empty(self.n_facilities, dtype=int)
        for i in range(self.n_facilities):
            if random.random() < 0.5:
                offspring1_solution[i] = parent1['solution'][i]
                offspring2_solution[i] = parent2['solution'][i]
            else:
                offspring1_solution[i] = parent2['solution'][i]
                offspring2_solution[i] = parent1['solution'][i]
        return {'solution': offspring1_solution, 'fitness': None}, {'solution': offspring2_solution, 'fitness': None}
    
    def _crossover_operator(self, parent1, parent2):
        if self.crossover_type == 'uniform':
            return self._uniform_crossover(parent1, parent2)
        elif self.crossover_type == 'one_point':
            offspring1, offspring2 = self._one_point_crossover(parent1, parent2)
            return offspring1, offspring2
        else:
            raise ValueError("Unknown crossover type")

    def _one_point_crossover(self, parent1, parent2):
        """
        Performs one-point crossover between two parents.
        """
        crossover_point = random.randint(1, self.n_facilities - 1)
        offspring1_solution = np.concatenate((parent1['solution'][:crossover_point], parent2['solution'][crossover_point:]))
        offspring2_solution = np.concatenate((parent2['solution'][:crossover_point], parent1['solution'][crossover_point:]))
        return {'solution': offspring1_solution, 'fitness': None}, {'solution': offspring2_solution, 'fitness': None}

    def _mutate(self, individual):
        """
        Performs bit-flip mutation on an individual.
        """
        mutated_solution = individual['solution'].copy()
        for i in range(self.n_facilities):
            if random.random() < self.mutation_rate:
                mutated_solution[i] = 1 - mutated_solution[i]
        return {'solution': mutated_solution, 'fitness': None}
    
    def _replacement(self, new_population):
        """
        Replaces the old population with a new population using elitism.
        """
        for individual in new_population:
            if individual['fitness'] is None:
                individual['fitness'] = self._calculate_fitness(individual)
        new_population.append({'solution': self.best_solution.copy(), 'fitness': self.best_fitness})
        new_population.sort(key=lambda ind: ind['fitness'])
        self.population = new_population[:self.population_size]

    def run(self):
        """
        Runs the genetic algorithm.

        Returns:
            tuple: Best solution, best fitness, fitness history, execution time, number of generations.
        """
        start_time = time.time()
        self._initialize_population()

        for generation in range(self.max_generations):
            self.generation = generation
            new_population = []
            while len(new_population) < self.population_size - 1:
                parent1 = self._tournament_selection()
                parent2 = self._tournament_selection()
                if random.random() < self.crossover_rate:
                    offspring1, offspring2 = self._crossover_operator(parent1, parent2)
                    new_population.append(self._mutate(offspring1))
                    new_population.append(self._mutate(offspring2))
                else:
                    new_population.append(self._mutate({'solution': parent1['solution'].copy(), 'fitness': None}))
                    new_population.append(self._mutate({'solution': parent2['solution'].copy(), 'fitness': None}))

            self._replacement(new_population)
            self._evaluate_population()
            self.fitness_history.append(self.best_fitness)

            # Optional convergence check
            if len(self.fitness_history) > 50 and np.std(self.fitness_history[-50:]) < 1e-6:
                print(f"Convergence reached at generation {generation + 1}")
                break

        end_time = time.time()
        return self.best_solution, self.best_fitness, self.fitness_history, end_time - start_time, self.generation + 1


In [ ]:
# --- 1. Data Input ---
file_name = 'UFLP-1.txt'

n_facilities, n_customers, fixed_costs, transport_costs = cargar_datos(file_name)

In [13]:
# --- 4. Experimental Setup ---
configurations = [
    {"population_size": 50, "mutation_rate": 0.1, "crossover_rate": 0.8, "tournament_size": 3, "max_generations": 100, "crossover_type": "uniform"},
    {"population_size": 100, "mutation_rate": 0.2, "crossover_rate": 0.7, "tournament_size": 5, "max_generations": 150, "crossover_type": "one_point"}
]
num_runs = 3
results = {}
output_dir = "uflp_ga_results"
os.makedirs(output_dir, exist_ok=True)
RANDOM_SEED = 42

print(f"Running GA with random seed: {RANDOM_SEED}\n")

for config in configurations:
    config_name = f"PopSize_{config['population_size']}_MutRate_{config['mutation_rate']}_CrossRate_{config['crossover_rate']}_TournSize_{config['tournament_size']}_MaxGen_{config['max_generations']}_{config['crossover_type']}"
    print(f"--- Running configuration: {config_name} ---")
    results[config_name] = []

    for run in range(num_runs):
        ga = UFLP_GA(n_facilities, n_customers, fixed_costs, transport_costs,
                     config["population_size"], config["mutation_rate"], config["crossover_rate"],
                     config["tournament_size"], config["max_generations"])
        if config["crossover_type"] == "uniform":
            ga._crossover_operator = ga._uniform_crossover
        elif config["crossover_type"] == "one_point":
            ga._crossover_operator = ga._one_point_crossover
        else:
            raise ValueError(f"Unknown crossover type: {config['crossover_type']}")

        best_solution, best_fitness, fitness_history, execution_time, num_generations = ga.run()
        results[config_name].append({
            "best_solution": best_solution,
            "best_cost": best_fitness,
            "execution_time": execution_time,
            "num_generations": num_generations,
            "fitness_history": fitness_history
        })
        print(f"Run {run + 1}/{num_runs}: Best Cost = {best_fitness:.2f}, Time = {execution_time:.2f}s, Generations = {num_generations}")
    print("\n")

Running GA with random seed: 42

--- Running configuration: PopSize_50_MutRate_0.1_CrossRate_0.8_TournSize_3_MaxGen_100_uniform ---
Run 1/3: Best Cost = 31582.00, Time = 5.81s, Generations = 100
Run 2/3: Best Cost = 28611.00, Time = 5.80s, Generations = 100
Run 3/3: Best Cost = 32823.00, Time = 6.20s, Generations = 100


--- Running configuration: PopSize_100_MutRate_0.2_CrossRate_0.7_TournSize_5_MaxGen_150_one_point ---
Convergence reached at generation 66
Run 1/3: Best Cost = 42184.00, Time = 10.10s, Generations = 66
Convergence reached at generation 137
Run 2/3: Best Cost = 35662.00, Time = 31.37s, Generations = 137
Run 3/3: Best Cost = 39326.00, Time = 24.06s, Generations = 150




In [14]:
# --- 5. Result Tables ---
summary_tables = {}
for config_name, run_results in results.items():
    costs = [res["best_cost"] for res in run_results]
    times = [res["execution_time"] for res in run_results]
    best_cost = min(costs)
    worst_cost = max(costs)
    mean_cost = np.mean(costs)
    std_dev_cost = np.std(costs)
    mean_time = np.mean(times)
    percent_deviation = (worst_cost - best_cost) / best_cost * 100 if best_cost != 0 else np.inf

    summary_tables[config_name] = pd.DataFrame({
        "Metric": ["Best Cost", "Worst Cost", "Mean Cost", "Std Dev Cost", "Mean Execution Time (s)", "Percent Deviation (%)"],
        "Value": [f"{best_cost:.2f}", f"{worst_cost:.2f}", f"{mean_cost:.2f}", f"{std_dev_cost:.2f}", f"{mean_time:.2f}", f"{percent_deviation:.2f}"]
    })

print("\n--- Summary Statistics per Configuration ---")
for config_name, summary_df in summary_tables.items():
    print(f"\nConfiguration: {config_name}")
    print(summary_df.to_string(index=False))
    summary_df.to_csv(os.path.join(output_dir, f"summary_{config_name}.csv"), index=False)


--- Summary Statistics per Configuration ---

Configuration: PopSize_50_MutRate_0.1_CrossRate_0.8_TournSize_3_MaxGen_100_uniform
                 Metric    Value
              Best Cost 28611.00
             Worst Cost 32823.00
              Mean Cost 31005.33
           Std Dev Cost  1767.23
Mean Execution Time (s)     5.94
  Percent Deviation (%)    14.72

Configuration: PopSize_100_MutRate_0.2_CrossRate_0.7_TournSize_5_MaxGen_150_one_point
                 Metric    Value
              Best Cost 35662.00
             Worst Cost 42184.00
              Mean Cost 39057.33
           Std Dev Cost  2669.36
Mean Execution Time (s)    21.84
  Percent Deviation (%)    18.29


In [18]:
# --- 6. Visualization ---
# Convergence Curves
plt.figure(figsize=(12, 6))
for config_name, run_results in results.items():
    for i, res in enumerate(run_results):
        plt.plot(res["fitness_history"], label=f"{config_name} (Run {i+1})" if i == 0 else "", alpha=0.3)
    min_length = min(len(res["fitness_history"]) for res in run_results)
    truncated_histories = [res["fitness_history"][:min_length] for res in run_results]
    avg_fitness_history = np.mean(truncated_histories, axis=0)
    plt.plot(avg_fitness_history, label=f"Avg. {config_name}", linewidth=2)

plt.xlabel("Generation")
plt.ylabel("Best Fitness")
plt.title("Convergence Curve of Best Fitness")
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, "convergence_curves.png"))
plt.close()

In [19]:
# Boxplots of Costs
cost_data = {config_name: [res["best_cost"] for res in run_results] for config_name, run_results in results.items()}
plt.figure(figsize=(10, 6))
sns.boxplot(data=pd.DataFrame(cost_data)).set(
    xlabel="Configuration",
    ylabel="Best Cost",
    title="Comparison of Best Cost Distributions"
)
plt.grid(True)
plt.savefig(os.path.join(output_dir, "cost_boxplots.png"))
plt.close()

In [20]:
# --- 7. Output and Documentation ---
print("\n--- Best Solutions Found ---")
for config_name, run_results in results.items():
    best_run = min(run_results, key=lambda x: x["best_cost"])
    open_facilities = np.where(best_run["best_solution"] == 1)[0] + 1 # Adjust index to match problem definition
    print(f"\nConfiguration: {config_name}")
    print(f"  Best Solution (Open Facilities): {open_facilities}")
    print(f"  Best Cost: {best_run['best_cost']:.2f}")

print(f"\nResults saved to directory: {output_dir}")
print(f"\nRandom seed used for reproducibility: {RANDOM_SEED}")


--- Best Solutions Found ---

Configuration: PopSize_50_MutRate_0.1_CrossRate_0.8_TournSize_3_MaxGen_100_uniform
  Best Solution (Open Facilities): [11 12 14 30 32 33]
  Best Cost: 28611.00

Configuration: PopSize_100_MutRate_0.2_CrossRate_0.7_TournSize_5_MaxGen_150_one_point
  Best Solution (Open Facilities): [12 17 32 40 44 49]
  Best Cost: 35662.00

Results saved to directory: uflp_ga_results

Random seed used for reproducibility: 42
